In [1]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

C:\Users\TOSHIBA\AppData\Local\Temp\ipykernel_6648\1855361594.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\TOSHIBA\AppData\Local\Temp\ipykernel_6648\1855361594.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_value = spacex_df['Payload Mass (kg)'].max()
min_value = spacex_df['Payload Mass (kg)'].min()
data  = spacex_df
total_s = spacex_df.groupby('Launch Site')['class'].sum()
total_s

Launch Site
CCAFS LC-40      7
CCAFS SLC-40     3
KSC LC-39A      10
VAFB SLC-4E      4
Name: class, dtype: int64

In [6]:
opts1 = [{'label':"All sites", 'value':"ALL"}]
for i in spacex_df['Launch Site'].unique():
    m = {'label':i, 'value':i}
    opts1.append(m)
# opts1
spacex_df.columns

Index(['Unnamed: 0', 'Flight Number', 'Launch Site', 'class',
       'Payload Mass (kg)', 'Booster Version', 'Booster Version Category'],
      dtype='object')

In [7]:
# Create an app layout
app = dash.Dash(__name__)
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                            options=opts1,
                                             value='ALL',
                                             placeholder='Select site',
                                            searchable=True
                                
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                # Function decorator to specify function input and output
                                
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                min=0, max=10000, step=1000,
                                marks={0: '0',
                                       100: '100'},
                                value=[min_value, max_value]),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        total_s = spacex_df.groupby('Launch Site')['class'].sum()
        fig = px.pie(total_s, values='class', 
        title='All Sites Success')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site].groupby('class')['class'].value_counts()
        fig = px.pie(filtered_df, values='class', 
        title='title')
        return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id="payload-slider", component_property="value")])
def get_scatter(site, r):
    space_df = spacex_df[(spacex_df['Payload Mass (kg)']>=r[0]) & (spacex_df['Payload Mass (kg)']<=r[1])]
    if site=='ALL':
        fig =px.scatter(space_df ,
            x = 'Payload Mass (kg)',
                    y='class',
                        color='Booster Version Category'
        )
        return fig
    
    else:
        filtered_df = space_df[spacex_df['Launch Site'] == site]
        fig =px.scatter(filtered_df ,
            x = 'Payload Mass (kg)',
                    y='class',
                        color='Booster Version Category'
        )
        return fig
# Run the app
if __name__ == '__main__':
    app.run_server()